In [13]:
import pandas as pd
from datetime import timedelta
from datetime import datetime
import json
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [14]:
# Reading the Excel file
path = "Book1.xlsx"
df = pd.read_excel(path, names=['Name', 'Roles', 'Responsibility', 'Time in', 'Time out'])

# Filter the dataframe based on 'Responsibility' and 'Name'
filtered_df = df[~df['Responsibility'].isin(['Technology', 'Office Work']) & ~df['Name'].str.contains('Available')]

# Get unique names
unique_names = filtered_df['Name'].unique()

# Create a dictionary with names as keys and 0 as the initial value
greeter_shift_done_dict = {name: 0 for name in unique_names}

# Display the initialized dictionary
greeter_shift_done_dict


{'Andrew (AJ) # Bowlen': 0,
 'Analuisa Flores Teran': 0,
 'Finn Broderick': 0,
 'Ricardo Gomez': 0,
 'Solymar Kneale': 0,
 'Carson Turk': 0,
 'Dev Mahajan': 0,
 'Ethan Palmer': 0,
 'Ana* Gonzalez': 0,
 'Marsailles Wesley': 0,
 'Josiah Galvan-Lewis': 0,
 'Kaeden Stander': 0,
 'Hanah # Wilkins': 0,
 'Kevin^ Jenkins': 0,
 'Luke* Ruhl': 0,
 'Joaquin Mendoza': 0,
 'Miguel Tanner': 0,
 'Elijah Kodjak': 0,
 'Lauren* Koski': 0,
 'Cristian Andrade': 0,
 'Lulu (Lyndsey)* Arterburn': 0,
 'AJ # Alkhamees': 0,
 'Joaquin Salinas': 0,
 'Cassidy Jacobs': 0,
 'Mariana Cepeda': 0,
 'Scott Hackney': 0,
 'Summer Sutton': 0,
 'Anahi Guitierrez': 0,
 'Diego Navarro': 0,
 'Lohith Ramesh': 0,
 'Saaijeesh Sottalu Naresh': 0,
 'Cassidy Sylves': 0,
 'Zadie Knapp': 0,
 'Serenity* Munoz': 0,
 'Elizabeth Jurry': 0,
 'Madelyn* Ozdarski': 0,
 'Naman Talwar': 0,
 'Aaron* Dorrance': 0,
 'Calvin # Levy': 0,
 'Ellia* Bono': 0,
 'Kayhaan Rashiq': 0,
 'Mali Chavez': 0}

In [15]:
# #converting workstatus.ipynb into .py file
# !jupyter nbconvert --to script work_status.ipynb

In [16]:
#imported the dataframe from workstatus notebook file
from work_status import work_status_df  

work_status_copy_df = work_status_df
work_status_copy_df.head(5)

,Name,Start_time,End_time,Working Flag
0,Andrew (AJ) # Bowlen,08:30:00,09:00:00,1
1,Andrew (AJ) # Bowlen,09:00:00,09:30:00,1
2,Andrew (AJ) # Bowlen,09:30:00,10:00:00,1
3,Andrew (AJ) # Bowlen,10:00:00,10:30:00,1
4,Andrew (AJ) # Bowlen,10:30:00,11:00:00,1


In [17]:
# Convert Start_time and End_time to datetime
work_status_copy_df['Start_time'] = pd.to_datetime(work_status_copy_df['Start_time'], format='%H:%M:%S').dt.time
work_status_copy_df['End_time'] = pd.to_datetime(work_status_copy_df['End_time'], format='%H:%M:%S').dt.time

# Find the minimum start time and maximum end time
min_start_time = min(work_status_copy_df['Start_time'])
max_end_time = max(work_status_copy_df['End_time'])

# Create 30-minute time intervals from min to max
time_intervals = []
current_time = pd.to_datetime(f"{min_start_time}")  # Use a reference date for time calculations

while current_time.time() < max_end_time:  # Change to < to prevent going over
    next_time = (current_time + timedelta(minutes=30)).time()
    
    # Adjust the end time to not exceed max_end_time
    if next_time > max_end_time:
        next_time = max_end_time
    
    time_intervals.append((current_time.time(), next_time))
    
    # Stop if next_time reaches or exceeds max_end_time
    if next_time == max_end_time:
        break
    
    current_time += timedelta(minutes=30)

# Create a results DataFrame to hold the output
results = []

# Iterate over the created time intervals
for start_time, end_time in time_intervals:
    # Filter employees who are working in the current time interval
    working_employees = work_status_copy_df[
        (work_status_copy_df['Start_time'] < end_time) & 
        (work_status_copy_df['End_time'] > start_time) &
        (work_status_copy_df['Working Flag'] == 1)
    ]

    # If there are working employees, add their names to the results, each on a new line
    if not working_employees.empty:
        for name in working_employees['Name'].unique():
            results.append({
                'Start_time': start_time,
                'End_time': end_time,
                'Employee': name
            })
    else:
        results.append({
            'Start_time': start_time,
            'End_time': end_time,
            'Employee': ''  
        })

# Create a DataFrame from the results
greeter_priority_df = pd.DataFrame(results)

# Display the result
greeter_priority_df

,Start_time,End_time,Employee
0,08:30:00,09:00:00,Andrew (AJ) # Bowlen
1,08:30:00,09:00:00,Analuisa Flores Teran
2,08:30:00,09:00:00,Finn Broderick
3,08:30:00,09:00:00,Ricardo Gomez
4,08:30:00,09:00:00,Solymar Kneale
...,...,...,...
209,18:00:00,18:30:00,Aaron* Dorrance
210,18:00:00,18:30:00,Calvin # Levy
211,18:00:00,18:30:00,Ellia* Bono
212,18:00:00,18:30:00,Kayhaan Rashiq


In [18]:
emp_count_req= pd.read_excel("00_Input/02_Emp_Count_Requirement.xlsx")
emp_count_req.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed
0,07:30:00,08:00:00,3,3,0,0,6
1,08:00:00,08:30:00,3,3,0,0,6
2,08:30:00,09:00:00,3,3,1,1,8
3,09:00:00,09:30:00,3,3,1,1,8
4,09:30:00,10:00:00,3,3,1,1,8


In [19]:
greeter_assignment=emp_count_req[['From_Time','To_Time','Greeter_Down_Needed','Greeter_Up_Needed']]
greeter_assignment.head()

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed
0,07:30:00,08:00:00,0,0
1,08:00:00,08:30:00,0,0
2,08:30:00,09:00:00,1,1
3,09:00:00,09:30:00,1,1
4,09:30:00,10:00:00,1,1


In [20]:
# Ensure all time columns are converted to strings in case they are of type datetime.time
greeter_priority_df['Start_time'] = greeter_priority_df['Start_time'].astype(str)
greeter_priority_df['End_time'] = greeter_priority_df['End_time'].astype(str)
work_status_copy_df['Start_time'] = work_status_copy_df['Start_time'].astype(str)
work_status_copy_df['End_time'] = work_status_copy_df['End_time'].astype(str)

# Function to calculate remaining working hours
def calculate_remaining_hours(employee, current_time, work_status_df):
    # Filter the work_status_copy_df for the specific employee and working hours
    employee_schedule = work_status_df[(work_status_df['Name'] == employee) & (work_status_df['Working Flag'] == 1)]
    
    remaining_time = 0
    current_time = datetime.strptime(current_time, '%H:%M:%S')  # Convert to datetime object
    
    for _, row in employee_schedule.iterrows():
        shift_start = datetime.strptime(row['Start_time'], '%H:%M:%S')
        shift_end = datetime.strptime(row['End_time'], '%H:%M:%S')
        
        if shift_start >= current_time:
            remaining_time += (shift_end - shift_start).total_seconds() / 3600  # Convert seconds to hours
    
    return remaining_time

# Add 'Remaining_hours_left' column to greeter_priority_df
greeter_priority_df['Remaining_hours_left'] = greeter_priority_df.apply(
    lambda row: calculate_remaining_hours(row['Employee'], row['End_time'], work_status_copy_df), axis=1
)

# Display the updated dataframe
greeter_priority_df.head(15)

,Start_time,End_time,Employee,Remaining_hours_left
0,08:30:00,09:00:00,Andrew (AJ) # Bowlen,5.0
1,08:30:00,09:00:00,Analuisa Flores Teran,2.0
2,08:30:00,09:00:00,Finn Broderick,2.0
3,08:30:00,09:00:00,Ricardo Gomez,3.0
4,08:30:00,09:00:00,Solymar Kneale,2.5
5,09:00:00,09:30:00,Andrew (AJ) # Bowlen,4.5
6,09:00:00,09:30:00,Analuisa Flores Teran,1.5
7,09:00:00,09:30:00,Finn Broderick,1.5
8,09:00:00,09:30:00,Ricardo Gomez,2.5
9,09:00:00,09:30:00,Solymar Kneale,2.0


In [21]:
# Convert 'Start_time' and 'End_time' to time objects in the main dataframe
greeter_priority_df['Start_time'] = pd.to_datetime(greeter_priority_df['Start_time'], format='%H:%M:%S').dt.time
greeter_priority_df['End_time'] = pd.to_datetime(greeter_priority_df['End_time'], format='%H:%M:%S').dt.time

def assign_priority(df, filtered_df, greeter_shift_done_dict):
    # Initialize the priority column
    df['Priority'] = 0  # Initialize the Priority column with zeros
    
    # Group by Start_time and End_time
    for (start, end), group in df.groupby(['Start_time', 'End_time']):
            group_with_non_zero_hours = group[group['Remaining_hours_left'] > 0].sort_values(by='Remaining_hours_left', ascending=True)
            group_with_zero_hours = group[group['Remaining_hours_left'] == 0]
            if not group_with_non_zero_hours.empty:
                group_with_non_zero_hours['Priority'] = group_with_non_zero_hours['Remaining_hours_left'].rank(method='dense', ascending=True).astype(int)
                df.loc[group_with_non_zero_hours.index, 'Priority'] = group_with_non_zero_hours['Priority']
            if not group_with_zero_hours.empty:
                max_priority = group_with_non_zero_hours['Priority'].max() if not group_with_non_zero_hours.empty else 1
                group_with_zero_hours['Priority'] = max_priority + 1
                df.loc[group_with_zero_hours.index, 'Priority'] = group_with_zero_hours['Priority']

    return df  # Return the modified DataFrame with the Priority column


In [10]:
def process_time_periods(greeter_priority_df, greeter_assignment, filtered_df, greeter_shift_done_dict):
    
    # Assign priorities before processing time periods
    greeter_priority_df = assign_priority(greeter_priority_df, filtered_df, greeter_shift_done_dict)

    time_periods = greeter_assignment[['From_Time', 'To_Time', 'Greeter_Up_Needed', 'Greeter_Down_Needed']].drop_duplicates()

    for idx, period in time_periods.iterrows():
        # Extract needed counts for upstairs and downstairs greeters
        up_needed = int(period['Greeter_Up_Needed'])
        down_needed = int(period['Greeter_Down_Needed'])

        current_period_data = greeter_priority_df[
            (greeter_priority_df['Start_time'] == period['From_Time']) &
            (greeter_priority_df['End_time'] == period['To_Time'])
        ]

        # Debug: Check current period data
        print(f"Current Period: {period['From_Time']} to {period['To_Time']}")
        print("Current Period Data:\n", current_period_data)

        # Sort employees by priority
        sorted_employees = current_period_data.sort_values('Priority')

        # Assign upstairs greeters based on needed count
        upstairs_greeters = sorted_employees.head(up_needed)['Employee'].tolist()
        print("Upstairs Greeters Assigned:", upstairs_greeters)

        # Filter out already assigned upstairs greeters for downstairs assignments
        remaining_employees = sorted_employees[~sorted_employees['Employee'].isin(upstairs_greeters)]

        # Assign downstairs greeters based on needed count
        downstairs_greeters = remaining_employees.head(down_needed)['Employee'].tolist() if down_needed > 0 else []
        print("Downstairs Greeters Assigned:", downstairs_greeters)
        print("-----------------------------------------------------------------------------")

        # Update the greeter_assignment DataFrame with assigned greeters
        greeter_assignment.at[idx, 'Upstairs Greeter'] = upstairs_greeters[0] if upstairs_greeters else None
        greeter_assignment.at[idx, 'Downstairs Greeter'] = downstairs_greeters[0] if downstairs_greeters else None

        # Update shift counts for assigned employees
        for employee in upstairs_greeters + downstairs_greeters:
            if employee:  # Only update for actual assignments
                greeter_shift_done_dict[employee] += 1

        # Update the priorities for assigned greeters
        for employee in upstairs_greeters + downstairs_greeters:
            if employee:  # Only update for actual assignments
                greeter_priority_df.loc[greeter_priority_df['Employee'] == employee, 'Priority'] = greeter_priority_df['Priority'].max() + 1

    return greeter_assignment, greeter_shift_done_dict

# Example call to the function
greeter_assignment, greeter_shift_done_dict = process_time_periods(
    greeter_priority_df, 
    greeter_assignment, 
    filtered_df, 
    greeter_shift_done_dict
)

# Output results
greeter_assignment.head(50)

Current Period: 07:30:00 to 08:00:00
Current Period Data:
 Empty DataFrame
Columns: [Start_time, End_time, Employee, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:00:00 to 08:30:00
Current Period Data:
 Empty DataFrame
Columns: [Start_time, End_time, Employee, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:30:00 to 09:00:00
Current Period Data:
   Start_time  End_time               Employee  Remaining_hours_left  Priority
0   08:30:00  09:00:00   Andrew (AJ) # Bowlen                   5.0         4
1   08:30:00  09:00:00  Analuisa Flores Teran                   2.0         1
2   08:30:00  09:00:00         Finn Broderick                   2.0         1
3   08:30:00  09:00:00     

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed,Upstairs Greeter,Downstairs Greeter
0,07:30:00,08:00:00,0,0,None,None
1,08:00:00,08:30:00,0,0,None,None
2,08:30:00,09:00:00,1,1,Analuisa Flores Teran,Finn Broderick
3,09:00:00,09:30:00,1,1,Solymar Kneale,Ricardo Gomez
4,09:30:00,10:00:00,1,1,Ethan Palmer,Carson Turk
5,10:00:00,10:30:00,1,1,Marsailles Wesley,Ana* Gonzalez
6,10:30:00,11:00:00,1,1,Kaeden Stander,Andrew (AJ) # Bowlen
7,11:00:00,11:30:00,1,1,Kevin^ Jenkins,Luke* Ruhl
8,11:30:00,12:00:00,1,1,Miguel Tanner,Joaquin Mendoza
9,12:00:00,12:30:00,1,1,Lauren* Koski,Elijah Kodjak


In [11]:
greeter_shift_done_dict


{'Andrew (AJ) # Bowlen': 1,
 'Analuisa Flores Teran': 1,
 'Finn Broderick': 1,
 'Ricardo Gomez': 1,
 'Solymar Kneale': 1,
 'Carson Turk': 1,
 'Dev Mahajan': 0,
 'Ethan Palmer': 1,
 'Ana* Gonzalez': 1,
 'Marsailles Wesley': 1,
 'Josiah Galvan-Lewis': 1,
 'Kaeden Stander': 1,
 'Hanah # Wilkins': 1,
 'Kevin^ Jenkins': 1,
 'Luke* Ruhl': 1,
 'Joaquin Mendoza': 1,
 'Miguel Tanner': 1,
 'Elijah Kodjak': 1,
 'Lauren* Koski': 1,
 'Cristian Andrade': 1,
 'Lulu (Lyndsey)* Arterburn': 1,
 'AJ # Alkhamees': 1,
 'Joaquin Salinas': 0,
 'Cassidy Jacobs': 1,
 'Mariana Cepeda': 1,
 'Scott Hackney': 1,
 'Summer Sutton': 1,
 'Anahi Guitierrez': 1,
 'Diego Navarro': 1,
 'Lohith Ramesh': 1,
 'Saaijeesh Sottalu Naresh': 1,
 'Cassidy Sylves': 1,
 'Zadie Knapp': 1,
 'Serenity* Munoz': 1,
 'Elizabeth Jurry': 1,
 'Madelyn* Ozdarski': 1,
 'Naman Talwar': 1,
 'Aaron* Dorrance': 1,
 'Calvin # Levy': 1,
 'Ellia* Bono': 1,
 'Kayhaan Rashiq': 1,
 'Mali Chavez': 1}